# Adding AgentOps to an existing Autogen service
<img src="https://raw.githubusercontent.com/AgentOps-AI/agentops/main/docs/images/external/autogen/autogen_integration.png?raw=true"/>

To get started, you'll need to install the AgentOps package and [set an API key](app.agentops.ai).

AgentOps automatically configures itself when it's initialized meaning your agent run data will be tracked and logged to your AgentOps account right away.

In [1]:
import agentops

from autogen import ConversableAgent, UserProxyAgent, config_list_from_json

# When initializing AgentOps, you can pass in optional tags to help filter sessions
agentops.init(api_key="...", tags=["simple-autogen-example"])

print('AgentOps is now running. You can view your session in the link above')

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=24c5d9f6-fb82-41e6-a468-2dc74a5318a3


AgentOps is now running. You can view your session in the link above


AutoGen will now start automatically tracking

* LLM prompts and completions
* Token usage and costs
* Agent names and actions
* Correspondence between agents
* Tool usage
* Errors

In [2]:

# Define an openai api key for the agent configurations
openai_api_key = "..."
config_list = [
    {
        "model": "gpt-4-turbo",
        "api_key": openai_api_key,
        "tags": ["gpt-4", "tool"]
    }
]

# Create the agent that uses the LLM.
assistant = ConversableAgent("agent", llm_config={"config_list": config_list})

# Create the agent that represents the user in the conversation.
user_proxy = UserProxyAgent("user", code_execution_config=False)

# Let the assistant start the conversation.  It will end when the user types "exit".
assistant.initiate_chat(user_proxy, message="How can I help you today?")

# Close your AgentOps session to indicate that it completed.
agentops.end_session("Success")
print('Success! Visit your AgentOps dashboard to see the replay')

agent (to user):

How can I help you today?

--------------------------------------------------------------------------------


Provide feedback to agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Tell me a joke about AgentOps


user (to agent):

Tell me a joke about AgentOps

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent (to user):

Why don't AgentOps teams ever play hide and seek?

Because good luck hiding when they always know where everyone is supposed to be!

--------------------------------------------------------------------------------


Provide feedback to agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Another


user (to agent):

Another

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent (to user):

Why did the AgentOps team bring a ladder to work?

Because they’re always reaching for high-level optimizations!

--------------------------------------------------------------------------------


Provide feedback to agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


🖇 AgentOps: This run's cost $0.001080
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=24c5d9f6-fb82-41e6-a468-2dc74a5318a3


Success! Visit your AgentOps dashboard to see the replay


You can view data on this run at [app.agentops.ai](app.agentops.ai).

The dashboard will display LLM events for each message sent by each agent, including those made by the human user.